In [1]:
#Install all the Reqiuired Libraries and Packages 
import pandas as pd               
import numpy as np                
import pyaudio
import librosa                    
import soundfile
import wave
import pickle
from sys import byteorder
from array import array
from struct import pack
from scipy.io import wavfile as wav
from sklearn.neural_network import MLPClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.
    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()


In [4]:
#Feature Extraction of Audio Files Function 
#Extract features (mfcc, chroma, mel) from a sound file

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [5]:
if __name__ == "__main__":
    # load the saved model (after training)
    model = pickle.load(open("Emotion_Voice_Detection_Model.pkl", "rb"))
    print("Please talk")
    filename = "livetest.wav"
    # record the file (start talking)
    record_to_file(filename)
    # extract features and reshape it
    features = extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1)
    # predict
    result = model.predict(features)[0]
    # show the result!
    print("result:", result)
   

Please talk
result: surprised


In [6]:
 import webbrowser
if result=='calm':
    webbrowser.open('https://www.youtube.com/watch?v=V1Pl8CzNzCw')
elif result=='neutral':
    webbrowser.open('https://www.youtube.com/watch?v=ork4qET7pV0')
elif result=='sad':
    webbrowser.open('https://www.youtube.com/watch?v=jJPMnTXl63E')
elif result=='angry':
    webbrowser.open('https://www.youtube.com/watch?v=8FAUEv_E_xQ')
elif result=='fearful':
    webbrowser.open('https://www.youtube.com/watch?v=8FAUEv_E_xQ')
elif result=='surprised':
    webbrowser.open('https://www.youtube.com/watch?v=VzppuKWR-5U')
elif result=='happy':
    webbrowser.open('https://www.youtube.com/watch?v=ZbZSe6N_BXs')
else:
    webbrowser.open('https://www.youtube.com/watch?v=XWGuzODhRNo&list=PLUv2Iww2TzxRAtFgzEH9hM62rrZjFxoKr&index=3&t=2682s')

# SampleTests 

In [73]:
from playsound import playsound
import playsound as ps
Audio="C:\\Users\\sunka\\Project Folder\\livetest.wav"
print(Audio)
ps.playsound("livetest.wav")


2022-04-22 13:21:05.549 INFO    playsound: Made a temporary copy of C:\Users\sunka\Project Folder\livetest.wav at C:\Users\sunka\AppData\Local\Temp\PSxxjkwzmn.wav - use other filenames with only safe characters to avoid this.


In [65]:
from pathlib import Path

from playsound import playsound

audio = Path().cwd() / "livetest.wav"
playsound(audio)

2022-04-22 13:15:28.721 INFO    playsound: Made a temporary copy of C:\Users\sunka\Project folder\livetest.wav at C:\Users\sunka\AppData\Local\Temp\PS6ze3r6ew.wav - use other filenames with only safe characters to avoid this.


In [74]:
import pyaudio
from IPython.display import Audio
Audio('livetest.wav')